# Class ThrDGuide

In [ ]:
import decimal
import math
import PythonQt
# from __main__ import qt, ctk, slicer
import numpy as np
import vtk

### 颜色德芙

In [4]:
def myColor(colorName):
  if colorName == "red":
    colorArr = [1,0,0]
  elif colorName == "green":
    colorArr = [0,1,0]
  elif colorName == "blue":
    colorArr = [0,0,1]
  elif colorName == "black":
    colorArr = [0,0,0]
  elif colorName == "white":
    colorArr = [1,1,1]
  elif colorName == "yellow":
    colorArr = [1,1,0]
  elif colorName == "pink":
    colorArr = [1,0,1]
  elif colorName == "cyan":
    colorArr = [0,1,1]
  return(colorArr)

### 皮肤德芙

In [5]:
# 皮肤模板

def skinSur(VolumeNode='CTchest', MiniThresd="150", Islands = 1, NodeName="Skintempt", color="yellow", opacity=.8):

  masterVolumeNode =slicer.util.getNode(VolumeNode)

  # Create segmentation
  segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
  segmentationNode.CreateDefaultDisplayNodes()  # only needed for display
  segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(masterVolumeNode)
  addedSegmentID = segmentationNode.GetSegmentation().AddEmptySegment("skin")

  # Create segment editor to get access to effects
  segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()
  segmentEditorWidget.setMRMLScene(slicer.mrmlScene)
  segmentEditorNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentEditorNode")
  segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)
  segmentEditorWidget.setSegmentationNode(segmentationNode)
  segmentEditorWidget.setMasterVolumeNode(masterVolumeNode)

  # Thresholding
  segmentEditorWidget.setActiveEffectByName("Threshold")
  effect = segmentEditorWidget.activeEffect()
  effect.setParameter("MinimumThreshold", MiniThresd)
  # effect.setParameter("MaximumThreshold","695")
  effect.self().onApply()

  # Islands
  if Islands == 1:
    segmentEditorWidget.setActiveEffectByName("Islands")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter("Operation", "KEEP_LARGEST_ISLAND")
    effect.self().onApply()
  # Dialate
  segmentEditorWidget.setActiveEffectByName("Margin")
  effect = segmentEditorWidget.activeEffect()
  effect.setParameter("MarginSizeMm", .1)
  effect.self().onApply()

  # # HoleFilling
  # segmentEditorWidget.setActiveEffectByName("Smoothing")
  # effect = segmentEditorWidget.activeEffect()
  # effect.setParameter("SmoothingMethod", "CLOSING")
  # effect.setParameter("KernalSizeMm", 4.00)
  # effect.self().onApply()

  # Smoothing
  # segmentEditorWidget.setActiveEffectByName("Smoothing")
  # effect = segmentEditorWidget.activeEffect()
  # effect.setParameter("SmoothingMethod", "MEDIAN")
  # effect.setParameter("KernelSizeMm", 11)
  # effect.self().onApply()

  # Clean up
  segmentEditorWidget = None
  slicer.mrmlScene.RemoveNode(segmentEditorNode)

  # Make segmentation results visible in 3D
  segmentationNode.CreateClosedSurfaceRepresentation()

  polyData = vtk.vtkPolyData()

  # Make sure surface mesh cells are consistently oriented
  surfaceMesh = segmentationNode.GetClosedSurfaceRepresentation(addedSegmentID, polyData)
  normals = vtk.vtkPolyDataNormals()
  normals.AutoOrientNormalsOn()
  normals.ConsistencyOn()
  normals.SetInputData(polyData)
  normals.Update()
  surfaceMesh = normals.GetOutput()

  vtkNode = slicer.modules.models.logic().AddModel(surfaceMesh)
  vtkNode.SetName(NodeName)

  modelDisplay = vtkNode.GetDisplayNode()
  modelDisplay.SetColor(myColor(color))  # yellow
  modelDisplay.SetOpacity(opacity)
  modelDisplay.SetBackfaceCulling(0)
  # modelDisplay.SetVisibility(1)
  modelDisplay.SetSliceIntersectionVisibility(True)

  slicer.mrmlScene.RemoveNode(segmentationNode)
  return()

In [6]:
# skinSur(VolumeNode = 'CT-chest',MiniThresd = "100",NodeName = "Bonetempt1",color = "blue")

MRMLNodeNotFoundException: could not find nodes in the scene by name or id 'CT-chest'

### 三点更新切面

[Set slice position and orientation from 3 markup fiducials](https://www.slicer.org/wiki/Documentation/Nightly/ScriptRepository#Measure_angle_between_two_slice_planes)
通过三个点，使得Red面reform

In [23]:
# Update plane from fiducial points
def UpdateSlicePlane(param1=None, param2=None):
  # Get point positions as numpy array
  import numpy as np
  nOfFiduciallPoints = markups.GetNumberOfFiducials()
  if nOfFiduciallPoints < 3:
    return  # not enough points
  points = np.zeros([3,nOfFiduciallPoints])
  for i in range(0, nOfFiduciallPoints):
    markups.GetNthFiducialPosition(i, points[:,i])
  # Compute plane position and normal
  planePosition = points.mean(axis=1)
  planeNormal = np.cross(points[:,1] - points[:,0], points[:,2] - points[:,0])
  planeX = points[:,1] - points[:,0]
  sliceNode.SetSliceToRASByNTP(planeNormal[0], planeNormal[1], planeNormal[2],
                               planeX[0], planeX[1], planeX[2],
                               planePosition[0], planePosition[1], planePosition[2], 0)

# # Get markup node from scene
# sliceNode = slicer.app.layoutManager().sliceWidget('Red').mrmlSliceNode()
# markups = slicer.util.getNode('F')

# # Update slice plane manually
# UpdateSlicePlane()

# # Update slice plane automatically whenever points are changed
# markupObservation = [markups, markups.AddObserver(slicer.vtkMRMLMarkupsNode.PointModifiedEvent, UpdateSlicePlane, 2)]

# markupObservation[0].RemoveObserver(markupObservation[1]) # 停止更新

### 加点德芙

In [430]:
def AddPoint(xyz=[0,0,0],modelName="N",r = .5,color = "red"):
  Point = vtk.vtkSphereSource()
  Point.SetCenter(xyz)
  Point.SetRadius(r)
  #     Point.SetPhiResolution(36)
  #     Point.SetThetaResolution(36)

  # VtkMapperPoint = vtk.vtkPolyDataMapper()
  # VtkMapperPoint.SetInputConnection(VtkSourcePoint.GetOutputPort())

  # VtkActorPoint = vtk.vtkActor()
  # VtkActorPoint.SetMapper(VtkMapperPoint)
  # VtkActorPoint.GetProperty().SetColor(1.0, 0.0, 0.0)

  PointNode = slicer.modules.models.logic().AddModel(Point.GetOutputPort())
  # PointNode.SetColor(1.0, 0.0, 0.0)
  PointNode.SetName(modelName)
  modelDisplay = PointNode.GetDisplayNode()
  modelDisplay.SetColor(myColor(color)) # yellow
  modelDisplay.SetOpacity(.9)
  modelDisplay.SetBackfaceCulling(0)
  return

### 点对点棱柱

In [8]:
# class ReadPoint:        
# ### 点对点正棱柱
def p2pCyl(startPoint, endPoint, radius = 1, modName = "Cyl", plus = 5, Seg = 3, RotY = 0, TX = 0, color = "red",Opacity = 1):
  # Create a cylinder.
  # Cylinder height vector is (0,1,0).
  # Cylinder center is in the middle of the cylinder
  # 圆柱体为(0,1,0),中心=True
  cylinderSource = vtk.vtkCylinderSource()
  cylinderSource.SetRadius(radius)
  cylinderSource.SetResolution(Seg)

  # Generate a random start and end point
  # startPoint = [0] * 3
  # endPoint = [0] * 3

  rng = vtk.vtkMinimalStandardRandomSequence()
  rng.SetSeed(8775070)  # For testing.8775070

  # Compute a basis
  normalizedX = [0] * 3
  normalizedY = [0] * 3
  normalizedZ = [0] * 3

  # The X axis is a vector from start to end
  vtk.vtkMath.Subtract(endPoint, startPoint, normalizedX)
  length = vtk.vtkMath.Norm(normalizedX)+plus
  #     length = 20
  vtk.vtkMath.Normalize(normalizedX)

  # The Z axis is an arbitrary vector cross X
  arbitrary = [0] * 3
  for i in range(0, 3):
    rng.Next()
    arbitrary[i] = rng.GetRangeValue(-10, 10)
  vtk.vtkMath.Cross(normalizedX, arbitrary, normalizedZ)
  vtk.vtkMath.Normalize(normalizedZ)

  # The Y axis is Z cross X
  vtk.vtkMath.Cross(normalizedZ, normalizedX, normalizedY)
  matrix = vtk.vtkMatrix4x4()
  # Create the direction cosine matrix
  matrix.Identity()
  for i in range(0, 3):
    matrix.SetElement(i, 0, normalizedX[i])
    matrix.SetElement(i, 1, normalizedY[i])
    matrix.SetElement(i, 2, normalizedZ[i])
  # Apply the transforms
  transform = vtk.vtkTransform()
  transform.Translate(startPoint)  # translate to starting point
  transform.Concatenate(matrix)  # apply direction cosines
  transform.RotateZ(-90.0)  # align cylinder to x axis
  transform.Scale(1.0, length, 1.0)  # scale along the height vector
  transform.Translate(0, .5, 0)  # translate to start of cylinder
  transform.RotateY(RotY)
  transform.Translate(TX, 0, 0)

  # Transform the polydata
  transformPD = vtk.vtkTransformPolyDataFilter()
  transformPD.SetTransform(transform)
  transformPD.SetInputConnection(cylinderSource.GetOutputPort())

  stlMapper = vtk.vtkPolyDataMapper()
  stlMapper.SetInputConnection(transformPD.GetOutputPort())
  vtkNode = slicer.modules.models.logic().AddModel(transformPD.GetOutputPort())
  vtkNode.SetName(modName)
  modelDisplay = vtkNode.GetDisplayNode()
  modelDisplay.SetColor(myColor(color) )# yellow
  modelDisplay.SetOpacity(Opacity)
  modelDisplay.SetBackfaceCulling(0)

  return

### 求圆柱体坐标点

In [9]:

# ### 求圆柱体坐标点
# class ReadPoint:
def Pcoord(modName="CylR"):
  modelNode = slicer.util.getNode(modName) # get model"tempt'
  sr = modelNode.GetPolyData() # node to PolyData

  pxyz = [0, 0, 0]
  NumP = sr.GetNumberOfPoints()
  #     polydata = sr
  for i in range(NumP//2):
    sr.GetPoint(i, pxyz)
    if i == 0:

      Pxyz= np.array([pxyz])

    else:

      Pxyz= np.append(Pxyz, np.array([pxyz]), axis=0)
  #         slicer.mrmlScene.RemoveNode(modelNode)
  return (Pxyz)

### 延长线终点坐标Lpcoord(P0,P1,length=100)

In [446]:
def Lpcoord(P0,P1,length=100):
  p2pCyl(P0, P1, radius=0, modName="Cyl00", plus=length, Seg=1, RotY=0, TX=0, color="cyan", Opacity=1)
  data = Pcoord("Cyl00")[0]
  AddPoint(data)
  p2pline(P1,data)
  modN = slicer.util.getNode("Cyl00")
  slicer.mrmlScene.RemoveNode(modN)
  return data

### 连点成线 p2pline(P0, P1, modelName="Line")

In [280]:
# ### 在slicer中连点成线
# class ReadPoint:
def p2pline(P0, P1, modelName="Line"):
  points = vtk.vtkPoints()
  points.SetNumberOfPoints(2)
  points.SetPoint(0,   P0[0], P0[1], P0[2])
  points.SetPoint(1,   P1[0], P1[1], P1[2])
  line = vtk.vtkLineSource()
  line.SetPoints(points)
  lineNode = slicer.modules.models.logic().AddModel(line.GetOutputPort())
  lineNode.SetName(modelName)

### 射线sRayCast(p_source, p_target,modleName)

In [504]:
# class ReadPoint:    
def sRayCast(p_source, p_target,modleName):
  '''

  :param filename: STL file to perform ray casting on.
  :param p_source: first point
  :param p_target: second point
  :return: code --> 0 : No intersection.
  :return: code --> +1 : p_source lies OUTSIDE the closed surface.
  :return; code --> -1 : p_source lies INSIDE closed surface

  '''


  # 注释掉stlreader,直接调用slicer中的modle
  #     reader = vtk.vtkSTLReader()
  #     reader.SetFileName(filename)
  #     reader.Update()
  #     mesh = reader.GetOutput()

  modelNode = slicer.util.getNode(modleName)  # get model"tempt'
  mesh = modelNode.GetPolyData()  # node to PolyData


  obbtree = vtk.vtkOBBTree()
  obbtree.SetDataSet(mesh)
  obbtree.BuildLocator()
  pointsVTKIntersection = vtk.vtkPoints()
  code = obbtree.IntersectWithLine(p_source, p_target, pointsVTKIntersection, None)
  # Extracting data
  pointsVTKIntersectionData = pointsVTKIntersection.GetData()
  noPointsVTKIntersection = pointsVTKIntersectionData.GetNumberOfTuples()
  if code == 1:
    xP = pointsVTKIntersectionData.GetTuple3(0) # 2020年08月25日06:24:21 只取第一个点
  #         p2p = np.linalg.norm(xP-p_source)
  else:
    xP = 0

  #     xPL = np.array([np.hstack((xP,p2p))])


  #     for idx in range(noPointsVTKIntersection):
  #         _tup = pointsVTKIntersectionData.GetTuple3(idx)
  #         XP = _tup
  #     if code == 1:
  #         xP = XP[0]
  #         p2p = np.linalg.norm(xP-p_source) # 求距离
  #     else:
  #         xP = 0
  #         p2p = 0


  return xP


### 读取数据德芙Pdata(fidNode = "T", groups = 2)

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=[2, 1, 4, 3]))
fig.add_trace(go.Bar(y=[1, 4, 3, 2]))
fig.update_layout(title = 'Hello Figure')
fig.show()


In [519]:
# ### 读取"F"
def Pdata(fidNode = "T", groups = 2):
  # 定义fid列表
  fidList = slicer.util.getNode(fidNode)
  # 定义fid个数
  Pdata.numFids = fidList.GetNumberOfFiducials()
  # 读取每个fid点坐标,形成二维数组
  for i in range(Pdata.numFids):
    ras = [0,0,0]
    fidList.GetNthFiducialPosition(i,ras)
    if i == 0:
      Mdata = np.array([ras])
    else:
      Mdata = np.append(Mdata, np.array([ras]), axis = 0)
  #     logging.debug(Mdata)
  data=np.array(Mdata).ravel()
  zcount = int(data.size/6)
  data = np.array(data.reshape(zcount, groups, 3))
  #     ras = [-1,-1,1]*data # ras 坐标
  return(data)

### 矫正椎前点

In [ ]:
data2 = data
def centerP(Data):
	zhuitiNum = data2.shape[0]
  for i in range(0,zhuitiNum):
    P1 = data2[i,0]
    P2 = data2[i,1]
    P3 = data2[i,2]
    Pa = (P2 + P3) / 2
    #         clineL = np.linalg.norm(P3 - centerP)
    # AddPoint(centerP)
    P2P = np.linalg.norm(P3 - centerP)
    # 在这里矫正P3点于垂直平分线
    p2pCyl(centerP,P3,160,"cFace",-1*P2P,4,0,0,"pink",1)


    zz0 = sRayCast(P1, P1+[20,0,0], "cFace")
    if zz0 == np.array([]):
      zz = sRayCast(P1, P1+[-20,0,0], "cFace")

    else:
      zz = zz0
    AddPoint(zz)
    # 垂直平分线长度
    cpl = np.linalg.norm(centerP-zz)
    # 中截面
    cP = (centerP+zz)/2
    p2pCyl(cP, zz, radius=36, modName="cface0", plus=-8, Seg=4, RotY=0, TX=0,color="green", Opacity=1)

    p2pCyl(zz, centerP, radius=0, modName="Cyl00", plus=cpl*CL, Seg=1, RotY=0, TX=0, color="green", Opacity=1)


    #         p2pCyl(P2, centerP, radius=1, modName="Cyl00", plus=clineL, Seg=4, RotY=0, TX=-1, color="green", Opacity=1)
    data = Pcoord("Cyl00")[1]
    AddPoint(data2)
    # 垂直平分线长度
    if i == 0:
      cPl = np.array([zz])
    else:
      cPl = np.append([cPl,zz])
    #         modN = slicer.util.getNode("cFace")
    #         slicer.mrmlScene.RemoveNode(modN)
    #         modN = slicer.util.getNode("Cyl00")
    #         slicer.mrmlScene.RemoveNode(modN)
    if i == 0:
      Data = P2
      Data = np.append(Data,data2)
      Data = np.append(Data,P3)
      Data = np.append(Data,data2)
    else:
      Data = np.append(Data,P2)
      Data = np.append(Data,data2)
      Data = np.append(Data,P3)
      Data = np.append(Data,data2)

  Data = np.array(Data.reshape(Data.shape[0]//6,2,3))
  return(Data)

### 读取3点数据 Pdata3(fidNode = "T", CL = 0)

In [518]:
# ### 读取"F"
def Pdata3(fidNode = "T", CL = 0):
  data = Pdata(fidNode,3)
  #     ras = [-1,-1,1]*data # ras 坐标
  for i in range(0,data.shape[0]):
    #             for i in range(0,zcount):
    P1 = data[i,0]
    P2 = data[i,1]
    P3 = data[i,2]
    centerP = (P2 + P3) / 2
    #         clineL = np.linalg.norm(P3 - centerP)
    # AddPoint(centerP)
    P2P = np.linalg.norm(P3 - centerP)
    # 在这里矫正P3点于垂直平分线
    p2pCyl(centerP,P3,160,"cFace",-1*P2P,4,0,0,"pink",1)


    zz0 = sRayCast(P1, P1+[20,0,0], "cFace")
    if zz0 == np.array([]):
      zz = sRayCast(P1, P1+[-20,0,0], "cFace")

    else:
      zz = zz0
    AddPoint(zz)
    # 垂直平分线长度
    cpl = np.linalg.norm(centerP-zz)
    # 中截面
    cP = (centerP+zz)/2
    p2pCyl(cP, zz, radius=36, modName="cface0", plus=-8, Seg=4, RotY=0, TX=0,color="green", Opacity=1)

    p2pCyl(zz, centerP, radius=0, modName="Cyl00", plus=cpl*CL, Seg=1, RotY=0, TX=0, color="green", Opacity=1)


    #         p2pCyl(P2, centerP, radius=1, modName="Cyl00", plus=clineL, Seg=4, RotY=0, TX=-1, color="green", Opacity=1)
    data = Pcoord("Cyl00")[1]
    AddPoint(data)
    # 垂直平分线长度
    if i == 0:
      cPl = np.array([zz])
    else:
      cPl = np.append([cPl,zz])
    #         modN = slicer.util.getNode("cFace")
    #         slicer.mrmlScene.RemoveNode(modN)
    #         modN = slicer.util.getNode("Cyl00")
    #         slicer.mrmlScene.RemoveNode(modN)
    if i == 0:
      Data = P2
      Data = np.append(Data,data)
      Data = np.append(Data,P3)
      Data = np.append(Data,data)
    else:
      Data = np.append(Data,P2)
      Data = np.append(Data,data)
      Data = np.append(Data,P3)
      Data = np.append(Data,data)

  Data = np.array(Data.reshape(Data.shape[0]//6,2,3))
  return(Data)

### 显示钉

In [ ]:
def Screws(fidName="T",screwR=2):

  FidP = Pdata3(fidName)  # 获取数据坐标
  P0 = FidP[:,0,:]
  P1 = FidP[:,1,:]
  # P0 = FidP[0]
  fids = FidP.shape[0] # 组数
  Screwl = []
  for i in range(fids):
    PZ = P0[i]  # 最窄点
    PT = P1[i]  # 椎前点
    #     AddPoint(PZ,"PZ",3)
    P2P = np.linalg.norm(PT - PZ)  # 最窄的到椎前点的距离
    seg = 4
    Axis_p2 = Lpcoord(PT,PZ)
    AddPoint(Axis_p2)
    boneP = sRayCast(Axis_p2,PT, "Bonetempt")# 入骨点坐标

    bonep = "boneP%s" % i
    AddPoint(boneP,bonep,1) # 标记入骨点
    screwtip = sRayCast(boneP,PT,"cface0")
    screwl = np.linalg.norm(np.array([screwtip]) - np.array([boneP]))
    for ii in range (20, 75, 5):
      L = ii - screwl
      if 0< L <= 5:
        screwL=ii
    #             break
    Screwl.append(screwL)
    LL = np.linalg.norm(np.array([PT]) - np.array([boneP]))
    p2pCyl(boneP,PT,screwR,"screw%s"%i,screwL-LL,8,0,0,"red",1)
  #   modN = slicer.util.getNode("cface0")
  #   slicer.mrmlScene.RemoveNode(modN)



  return(Screwl)

In [ ]:
Screws("F")

### 沙盘测试

In [421]:
P2P = np.linalg.norm(P1 - P2)  # 最窄的到椎前点的距离
seg = 4
Axis_p2 = Lpcoord(P2,P1)
logging.debug(Axis_p2)
boneP = sRayCast(Axis_p2,P1, "Bonetempt")# 入骨点坐标
bonep = "boneP%s" % i

In [422]:
screwtip = sRayCast(boneP,P1,"cface0")
screwtip

(10.31633186340332, 211.78538513183594, -155.4287872314453)

In [400]:
screwl = np.linalg.norm(screwtip - boneP)

TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

In [426]:
AddPoint(P1)

In [281]:
p2pCyl(P1, P2, radius=0, modName="Cyl00", plus=length, Seg=1, RotY=0, TX=0, color="cyan", Opacity=1)
data = Pcoord("Cyl00")[0]
AddPoint(data)
p2pline(P1,data)
modN = slicer.util.getNode("Cyl00")
slicer.mrmlScene.RemoveNode(modN)

In [284]:
Screws

#### 横向调整
##### 取3个关键点（-。5--1--3）

In [482]:
Screws("F")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [478]:
data  = Pdata("F",3)
# data1 = Pdata("F",3)
# data2  = Pdata3("F",3)
# p2pline(data0[0,0],data1[0,0])
# p2pline(data2[0,0],data1[0,0])
#xx = Data.shape[0]

In [479]:
data

array([[[   5.2       ,  225.692     , -156.179     ],
        [  15.42888786,  197.87877371, -154.67856762],
        [  -4.18671396,  197.58600353, -154.67856762]]])

In [481]:
P2 = data[0,0]
P1 = data[0,1]

In [ ]:
sRayCast(Axis_p2,P1, "Bonetempt")

In [272]:

cpl = np.linalg.norm(centerP-zz) # 垂直平分线长度
X = np.linalg.norm(P2 - P3)*.5
Pc = .5*(P2+P3)
Y = np.linalg.norm(P1 - Pc)*.5
angleA = math.degrees(math.atan(Y/X))
# angleA
L = X*math.tan(math.radians(angleA+3))

p2pCyl(Pc,P1,0,"axis",L-cpl,1,0,0,"red",1)
cP = Pcoord("axis")
p2pCyl(cP,Pz,ScrewR,ScrewLebel,lengeh,6,0,0,"blue",1)


15.68925477633742

In [226]:
p2pCyl(zz, centerP, radius=0, modName="Cyl00", plus=cpl*1, Seg=1, RotY=0, TX=0, color="green", Opacity=1)

In [232]:
data = Pcoord("Cyl00")
AddPoint(data[1])

In [259]:
CL = [-.5,0,3]
for ii, v in enumerate(CL):
  data = Pdata3("F",v)
  for i in range(2):
    #     zuobiao = []
    P1 = data[i,0]
    P2 = data[i,1]
    no = ii*2
    No = "axis_%s"%no
    p2pCyl(P2,P1,2,No,20,4,0,0,"green",1)
    zuobiao = Pcoord(No)
    zhongdian = (zuobiao[0]+zuobiao[4])*.5
    AddPoint(zhongdian)
    if i == 0:
      ZB = np.array([zhongdian])
    else:
      ZB = np.append(ZB, np.array([zhongdian]), axis = 0)
  ZBB = np.array([])
  if ii == -.5:
    ZBB = np.array([ZB])
  else:
    ZBB = np.append(ZBB, np.array([ZB]), axis= 0)
Pl01=(ZBB[0,0]+ZBB[1,0])*.5
Pl12=(ZBB[2,0]+ZBB[1,0])*.5
Pr01=(ZBB[0,1]+ZBB[1,1])*.5
Pr12=(ZBB[2,1]+ZBB[1,1])*.5
Pl = Pdata3("F",0)[0,0]
Pr = Pdata3("F",0)[1,0]
p2pCyl(Pl01,Pl,2,"axis_%s"%1,20,4,0,0,"green",1)
p2pCyl(Pl12,Pl,2,"axis_%s"%3,20,4,0,0,"green",1)
p2pCyl(Pr01,Pr,2,"axis_%s"%1,20,4,0,0,"green",1)
p2pCyl(Pr12,Pr,2,"axis_%s"%3,20,4,0,0,"green",1)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 3 dimension(s)

In [100]:
CL = [-.5,0,3]
for ii, v in enumerate(CL):
  print(ii*2,v)

0 -0.5
2 0
4 3


In [89]:
ZBB.shape

(3, 2, 3)

#####  取P1中点

In [104]:
Node = getNode('bone')
DisplayNode = Node.GetDisplayNode()
DisplayNode.SetVisibility(True) # Hide all points

In [26]:
data01 = .5*(data0[0,0]+data1[0,0])
AddPoint(data01)

In [61]:
ZB

array([[  -9.72998238,  177.8343277 , -153.34186554],
       [  22.38219357,  179.47812653, -154.33469391]])

In [49]:
data1 = Data
data1

array([[[  -4.08214705,  197.00335719, -154.14831543],
        [   0.80931318,  309.55984497, -158.0579071 ]],

       [[  15.08734448,  198.09578304, -154.74470339],
        [   0.80931318,  309.55984497, -158.0579071 ]]])

##### 中切面

In [53]:
P1 = data1[0,0]
P2 = data1[1,0]
Pc = .5*(P1+P2)
P2P = np.linalg.norm(Pc - P1)

p2pCyl(Pc,P1,160,"cFace",-1*P2P,4,0,0,"pink",1)

In [26]:
PP0 = data1[0,1]
PP1 = PP0+[20,0,0]
PP11 = PP0+[-20,0,0]


# sRayCast(PP0, PP1,"cFace")

In [80]:
AddPoint(sRayCast(PP0, PP11,"cFace"))

In [37]:
def centerP():

NameError: name 'modleName' is not defined

### 做横切面
[Set slice position and orientation from 3 markup fiducials](https://www.slicer.org/wiki/Documentation/Nightly/ScriptRepository#Measure_angle_between_two_slice_planes)
通过三个点，使得Red面reform


In [60]:
P2P

9.604926836973958

In [ ]:
p2pCyl(centerP, P3, radius=1, modName="Cyl00", plus=clineL*CL, Seg=1, RotY=0, TX=-1, color="green", Opacity=1)

In [8]:
skinSur(VolumeNode='CT-chest', MiniThresd="-300", NodeName="Skintempt", color="white", opacity=.6)
skinSur(VolumeNode='CT-chest', MiniThresd="150", NodeName="Bonetempt", color="black", opacity=.1)

6 islands created (2096 ignored)
7 islands created (2068 ignored)


()

### 从坐标求面积

In [13]:
def vertebral_zD(P1,P2,P3):
  side0 = np.linalg.norm(P2-P1)
  side1 = np.linalg.norm(P3-P1)
  side2 = np.linalg.norm(P3-P2)
  # 利用海伦公式。s=sqr(p*(p-a)(p-b)(p-c))
  p = (side0+side1+side2)/2 # 半周长
  area = (p*(p-side0)*(p-side1)*(p-side2))**0.5
  return(area/side0*2)

In [14]:
area = TriArea(P1,P2,P3)
print(area)

32.620786858981674


### 尺子德芙

In [82]:
numFids

AttributeError: 'function' object has no attribute 'numFids'

In [12]:
p2pCyl(P1, P3, radius=1, modName="Cyl00", plus=100, Seg=2, RotY=0, TX=-1, color="green", Opacity=1)

TypeError: Subtract argument 1: expected a sequence of 3 values, got numpy.float64

In [47]:
clineL = np.linalg.norm(P3-centerP)

p2pCyl(zz[0], P1, radius = 1, modName = "Cyl00", plus = clineL, Seg = 3, RotY = 0, TX = -1, color = "green",Opacity = 1)

In [18]:
for i in range(0,data.shape[0]):
  pp0 = Pdata3()[i,0,:]
  pp1 = Pdata3()[i,1,:]
  p2pCyl(pp1, pp0, radius = 10, modName = "Cyl000", plus = 100, Seg = 1, RotY = 0, TX = -10,color = "green",Opacity = 1)
  p2pCyl(pp1, pp0, radius = 1, modName = "Cyl00", plus = 5, Seg = 3, RotY = 0, TX = -1,color = "blue",Opacity = .6)

In [65]:
Axis_p2[1]

array([-197.0597229 , -201.33206177,  159.77024841])

In [23]:
axis = sRayCast(axis_2P[1],axis_2P[0],"Skintempt")
print (axis)

((-35.292694091796875, -122.9182357788086, -54.0883674621582), 91.18938027606472)


In [33]:
axis[0]

(226.74679565429688, 69.61011505126953, -167.89523315429688)

In [32]:
skin2bone = sRayCast(axis[0], zz[2], "Bonetempt")

(226.74679565429688, 69.61011505126953, -167.89523315429688)

In [29]:
AddPoint(axis[0])

### 3点定三角

In [8]:
def triangle3P(p0,p1,p2,color = "green"):
  Points = vtk.vtkPoints()
  Triangles = vtk.vtkCellArray()

  Points.InsertNextPoint(p0)
  Points.InsertNextPoint(p1)
  Points.InsertNextPoint(p2)

  Triangle = vtk.vtkTriangle();
  Triangle.GetPointIds().SetId(0, 0);
  Triangle.GetPointIds().SetId(1, 1);
  Triangle.GetPointIds().SetId(2, 2);
  Triangles.InsertNextCell(Triangle);

  polydata = vtk.vtkPolyData()
  polydata.SetPoints(Points)
  polydata.SetPolys(Triangles)

  model = slicer.modules.models.logic().AddModel(polydata)

  # Tune display properties
  modelDisplay = model.GetDisplayNode()
  modelDisplay.SetColor(myColor(color)) # yellow
  modelDisplay.SetOpacity(0.5)
  modelDisplay.SetBackfaceCulling(0)




### 3点定平面grid

In [17]:
def plane3P(p0,p1,p2):
  # Create model node
  planeSource = vtk.vtkPlaneSource()
  planeSource.SetOrigin(p0)
  planeSource.SetPoint1(p1)
  planeSource.SetPoint2(p2)
  planeSource.SetXResolution(2)
  planeSource.SetYResolution(2)
  model = slicer.modules.models.logic().AddModel(planeSource.GetOutputPort())

  # Tune display properties
  modelDisplay = model.GetDisplayNode()
  modelDisplay.SetColor(0,0,1) # yellow
  modelDisplay.SetOpacity(0.5)
  modelDisplay.SetBackfaceCulling(0)

  # # Add texture (just use image of an ellipsoid)
  e = vtk.vtkImageGridSource()
  e.SetDataSpacing(5.0,5.0,5.0)
  modelDisplay.SetTextureImageDataConnection(e.GetOutputPort())

In [15]:
data1 = Pdata3("F", 0)
data1

array([[[  -1.14865005,  197.02999878, -152.56509399],
        [   4.06251001,  225.97920227, -156.38540649]],

       [[  11.29459953,  198.32299805, -155.06950378],
        [   4.06251001,  225.97920227, -156.38540649]]])

In [18]:
plane3P(data1[0,1],data1[0,0],data1[1,0])

### 大循环

In [ ]:
# ##大循环: 获取
def DoLoop(fidName):
  #     fidName="F"
  FidP =Pdata(fidName) # 获取数据
  #     P0 = FidP[0]
  fids = FidP.shape[0] # 组数
  #     skinSur('1020_2') # 大模板
  skinSur(VolumeNode = '1020_3',MiniThresd = "150",NodeName = "Bonetempt",color = "blue",opacity=1)
  skinSur(VolumeNode = '1020_3',MiniThresd = "-300",NodeName = "Skintempt",color = "yellow",opacity=.5)
  colors = ["red","green","white","cyan"]
  # XY = []
  for angle, color in enumerate(colors):
    roty = 90*angle
    for i in range(fids):
      PO = FidP[i][0] # 入皮点
      PT = FidP[i][1] # 靶点
      seg = 3
      modName = "N%s%s"%(angle,i)
      modN = "N"
      AddPoint(PO)
      Cyl12 =p2pCyl(PT,PO,10,modName,20,seg,roty) # 建一个3棱柱
      coord =Pcoord(modName) # 得到棱柱的坐标
      modNode =slicer.util.getNode(modName)
      slicer.mrmlScene.RemoveNode(modNode) # 删掉棱柱

      L = []
      XYZ = []
      for j in range(seg): # 棱柱坐标和模板的交点
        aaa = sRayCast(coord[2*j],coord[2*j+1],"Skintempt")
        leng = aaa[1]
        XYZ = aaa[0]
        AddPoint(XYZ)
        L = np.append(L, leng) # 长度
        if j == 0:
          xyz = np.array([XYZ])
        else:
          xyz = np.append(xyz, np.array([XYZ]),axis=0) # 交点坐标
      minL = min(L) # 求出长度最小值
      min_index = np.argmin(L) # ndarray最小值索引

      tabL = np.around(L-minL+10,2)
      tableNode=slicer.mrmlScene.AddNewNodeByClass("vtkMRMLTableNode")
      tableNode.SetName("xyzL_table")
      updateTableFromArray(tableNode, tuple(tabL))
      ml=slicer.modules.markups.logic()
      newLandmarksNodeID = ml.AddNewFiducialNode("Clock_%s%s"%(i,roty)) # 交点成组

      p2pline(PT, PO, modName)

      p2pline(xyz[min_index], PO, modName)

      triangle3P(xyz[0],xyz[1],xyz[2],color)
      # # # 每个循环截屏一次,无法同步
      # #             renderWindow = slicer.app.layoutManager().threeDWidget(0).threeDView().renderWindow()
      # #             renderWindow.SetAlphaBitPlanes(1)
      # #             wti = vtk.vtkWindowToImageFilter()
      # #             wti.SetInputBufferTypeToRGBA()
      # #             wti.SetInput(renderWindow)
      # #             writer = vtk.vtkPNGWriter()
      # #             writer.SetFileName("shortcut/screenshot_%s-%s.png"%(i,roty))
      # #             writer.SetInputConnection(wti.GetOutputPort())
      # #             writer.Write()



      for ii, v in enumerate(xyz):
        fidNode = slicer.mrmlScene.GetNodeByID(newLandmarksNodeID)
        n = fidNode.AddFiducial(v[0],v[1],v[2])
        fidNode.SetNthFiducialLabel(n, "%s"%tabL[ii])
      #         #     XYZ = []
      if i ==0:
        XY=[xyz]
        Length=np.array([tabL])
      else:
        XY=np.append(XY, [xyz],axis = 0)
        Length=np.append(Length, np.array([tabL]),axis = 0)
  #     data=np.array(Mdata).ravel()
  #     zcount = int(data.size/6)
  #     data = np.array(data.reshape(zcount, 2, 3))
  return ()
# renderWindow = slicer.app.layoutManager().threeDWidget(0).threeDView().renderWindow()
# renderWindow.SetAlphaBitPlanes(1)
# wti = vtk.vtkWindowToImageFilter()
# wti.SetInputBufferTypeToRGBA()
# wti.SetInput(renderWindow)
# writer = vtk.vtkPNGWriter()
# writer.SetFileName("screenshot.png")
# writer.SetInputConnection(wti.GetOutputPort())
# writer.Write()

In [ ]:
XYZL = DoLoop("F")

In [100]:
XYZL[1]

array([[[ 27.06436348,   5.98488712,  92.4755249 ],
        [ 19.32126999,  -3.46115136, 104.78932953],
        [ 13.99338531,  -4.77115917,  88.32833099]],

       [[-19.23415947,  -1.69604027,  86.92528534],
        [-19.88726425,  -2.53067756, 104.22116089],
        [-30.6495285 ,   8.43087006,  95.57241058]],

       [[-19.21258354,  -1.36595595,  71.66811371],
        [-19.2217598 ,  -1.87269425,  89.04682922],
        [-29.64843178,   9.3817997 ,  80.48151398]],

       [[-18.90452957,  -1.70819497,  55.7275238 ],
        [-18.77264977,  -1.83685625,  73.08963013],
        [-29.51259804,   9.06407166,  64.39335632]],

       [[ 25.60038567,   7.95990562,  63.94112778],
        [ 16.40051079,  -2.82455254,  74.15718079],
        [ 14.26823902,  -3.31291032,  56.97288132]],

       [[ 28.68374634,   8.90022755,  80.70050049],
        [ 18.38935089,  -2.3036015 ,  89.82970428],
        [ 16.94962502,  -1.50591063,  72.58277893]]])

### 三棱柱显示

In [26]:
fidName="T"
FidP =Pdata(fidName) # 获取数据
fids = FidP.shape[0] # 组数
skinSur(VolumeNode = 'pigsp',MiniThresd = "150",NodeName = "Bonetempt",color = "blue",opacity=1)
skinSur(VolumeNode = 'pigsp',MiniThresd = "-300",NodeName = "Skintempt",color = "yellow",opacity=.5)
colors = ["red","green","white","cyan"]

### 旋转三棱柱

In [27]:
# 需要先删除再做一个旋转的棱柱
roty = 90
for i in range(fids):
  PO = FidP[i][0] # 入皮点
  PT = FidP[i][1] # 靶点
  seg = 3
  modName = "N%s"%(i)
  #     AddPoint(PO)
  Cyl12 =p2pCyl(PT,PO,10,modName,10,seg,roty,"black",0.6) # 建一个3棱柱

### 获取落脚点

In [ ]:
coord =Pcoord(modName) # 得到棱柱的坐标
modNode =slicer.util.getNode(modName)
slicer.mrmlScene.RemoveNode(modNode) # 删掉棱柱

In [ ]:
L = []
XYZ = []
for j in range(seg): # 棱柱坐标和模板的交点
  aaa = sRayCast(coord[2*j],coord[2*j+1],"Skintempt")
  leng = aaa[1]
  XYZ = aaa[0]
  AddPoint(XYZ)
  L = np.append(L, leng) # 长度
  if j == 0:
    xyz = np.array([XYZ])
  else:
    xyz = np.append(xyz, np.array([XYZ]),axis=0) # 交点坐标
minL = min(L) # 求出长度最小值
min_index = np.argmin(L) # ndarray最小值索引

In [ ]:
tabL = np.around(L-minL+10,2)
tableNode=slicer.mrmlScene.AddNewNodeByClass("vtkMRMLTableNode")
tableNode.SetName("xyzL_table")
updateTableFromArray(tableNode, tuple(tabL))
ml=slicer.modules.markups.logic()
newLandmarksNodeID = ml.AddNewFiducialNode("Clock_%s%s"%(i,roty)) # 交点成组

In [ ]:
p2pline(PT, PO, modName)

In [ ]:
p2pline(xyz[min_index], PO, modName)

In [ ]:
triangle3P(xyz[0],xyz[1],xyz[2],color)
# # # 每个循环截屏一次,无法同步            
# #             renderWindow = slicer.app.layoutManager().threeDWidget(0).threeDView().renderWindow()
# #             renderWindow.SetAlphaBitPlanes(1)
# #             wti = vtk.vtkWindowToImageFilter()
# #             wti.SetInputBufferTypeToRGBA()
# #             wti.SetInput(renderWindow)
# #             writer = vtk.vtkPNGWriter()
# #             writer.SetFileName("shortcut/screenshot_%s-%s.png"%(i,roty))
# #             writer.SetInputConnection(wti.GetOutputPort())
# #             writer.Write()

In [ ]:
for ii, v in enumerate(xyz):
  fidNode = slicer.mrmlScene.GetNodeByID(newLandmarksNodeID)
  n = fidNode.AddFiducial(v[0],v[1],v[2])
  fidNode.SetNthFiducialLabel(n, "%s"%tabL[ii])
#         #     XYZ = []
if i ==0:
  XY=[xyz]
  Length=np.array([tabL])
else:
  XY=np.append(XY, [xyz],axis = 0)
  Length=np.append(Length, np.array([tabL]),axis = 0)

In [29]:
try:
  slicer.util.getNode("Bonetempt")
except:
  print("no")

In [ ]:
segmentationNode =slicer.util.getNode(('Segmentation')
segmentId = 'Segment_1'

# Get array voxel coordinates
import numpy as np
seg=arrayFromSegment(segmentation_node, segmentId)
# numpy array has voxel coordinates in reverse order (KJI instead of IJK)
# and the array is cropped to minimum size in the segmentation
mean_KjiCropped = [coords.mean() for coords in np.nonzero(seg)]

# Get segmentation voxel coordinates
segImage = segmentationNode.GetBinaryLabelmapRepresentation(segmentId)
segImageExtent = segImage.GetExtent()
# origin of the array in voxel coordinates is determined by the start extent
mean_Ijk = [mean_KjiCropped[2], mean_KjiCropped[1], mean_KjiCropped[0]] + np.array([segImageExtent[0], segImageExtent[2], segImageExtent[4]])

# Get segmentation physical coordinates
ijkToWorld = vtk.vtkMatrix4x4()
segImage.GetImageToWorldMatrix(ijkToWorld)
mean_World = [0, 0, 0, 1]
ijkToRas.MultiplyPoint(np.append(mean_Ijk,1.0), mean_World)
mean_World = mean_World[0:3]

# If segmentation node is transformed, apply that transform to get RAS coordinates
transformWorldToRas = vtk.vtkGeneralTransform()
slicer.vtkMRMLTransformNode.GetTransformBetweenNodes(segmentationNode.GetParentTransformNode(), None, transformWorldToRas)
mean_Ras = transformWorldToRas.TransformPoint(mean_World)

# Show mean position value and jump to it in all slice viewers
print(mean_Ras)
slicer.modules.markups.logic().JumpSlicesToLocation(mean_Ras[0], mean_Ras[1], mean_Ras[2], True)

In [33]:
# Inputs
volumeNode =slicer.util.getNode(('1020_2')
markupsNode =slicer.util.getNode(('F')

# Get voxel position in IJK coordinate system
import numpy as np
volumeArray = slicer.util.arrayFromVolume(volumeNode)
# Get position of highest voxel value
point_Kji = np.where(volumeArray == volumeArray.max())
point_Ijk = [point_Kji[2][0], point_Kji[1][0], point_Kji[0][0]]

# Get physical coordinates from voxel coordinates
volumeIjkToRas = vtk.vtkMatrix4x4()
volumeNode.GetIJKToRASMatrix(volumeIjkToRas)
point_VolumeRas = [0, 0, 0, 1]
volumeIjkToRas.MultiplyPoint(np.append(point_Ijk,1.0), point_VolumeRas)

# If volume node is transformed, apply that transform to get volume's RAS coordinates
transformVolumeRasToRas = vtk.vtkGeneralTransform()
slicer.vtkMRMLTransformNode.GetTransformBetweenNodes(volumeNode.GetParentTransformNode(), None, transformVolumeRasToRas)
point_Ras = transformVolumeRasToRas.TransformPoint(point_VolumeRas[0:3])

# Add a markup at the computed position and print its coordinates
markupsNode.AddFiducial(point_Ras[0], point_Ras[1], point_Ras[2], "max")
print(volumeIjkToRas)

vtkMatrix4x4 (0x7f9a15551dc0)
  Debug: Off
  Modified Time: 194825592
  Reference Count: 1
  Registered Events: (none)
  Elements:
    0.291016 0 0 -74.4893 
    0 0.291016 0 -52.0306 
    0 0 1.25 -65 
    0 0 0 1 




In [34]:
def planeFit(points):
  """
  p, n = planeFit(points)

  Given an array, points, of shape (d,...)
  representing points in d-dimensional space,
  fit an d-dimensional plane to the points.
  Return a point, p, on the plane (the point-cloud centroid),
  and the normal, n.
  """
  import numpy as np
  from numpy.linalg import svd
  points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
  assert points.shape[0] <= points.shape[1], "There are only {} points in {} dimensions.".format(points.shape[1], points.shape[0])
  ctr = points.mean(axis=1)
  x = points - ctr[:,np.newaxis]
  M = np.dot(x, x.T) # Could also use np.cov(x) here.
  return ctr, svd(M)[0][:,-1]

In [38]:
planeFit(XYZL[1][0])

(array([ -8.97545815, -13.21340942, -15.08805831]), array([-0.12095286, -0.25668639,  0.9588965 ]))

In [89]:
Points = vtk.vtkPoints()
Triangles = vtk.vtkCellArray()

Points.InsertNextPoint(1.0, 0.0, 0.0)
Points.InsertNextPoint(0.0, 0.0, 0.0)
Points.InsertNextPoint(0.0, 1.0, 0.0)

Triangle = vtk.vtkTriangle();
Triangle.GetPointIds().SetId(0, 0);
Triangle.GetPointIds().SetId(1, 1);
Triangle.GetPointIds().SetId(2, 2);
Triangles.InsertNextCell(Triangle);

#setup colors (setting the name to "Colors" is nice but not necessary)
Colors = vtk.vtkUnsignedCharArray();
Colors.SetNumberOfComponents(3);
Colors.SetName("Colors");
Colors.InsertNextTuple3(255,0,0);

polydata = vtk.vtkPolyData()
polydata.SetPoints(Points)
polydata.SetPolys(Triangles)

polydata.GetCellData().SetScalars(Colors);
polydata.Modified()
if vtk.VTK_MAJOR_VERSION <= 5:
  polydata.Update()

writer = vtk.vtkXMLPolyDataWriter();
writer.SetFileName("TriangleSolidColor.vtp");
if vtk.VTK_MAJOR_VERSION <= 5:
  writer.SetInput(polydata)
else:
  writer.SetInputData(polydata)
writer.Write()

1

In [54]:
def TriPrism(fidName = "F"):
  FidP = Pdata(fidName)  # 获取数据
  P0 = FidP[:,0,:]
  P1 = FidP[:,1,:]
  # P0 = FidP[0]
  fids = FidP.shape[0]  # 组数
  # skinSur('pigsp')  # 大模板
  # XY = []
  for i in range(fids):
    PO = P0[i] # 入皮点
    PT = P1[i]  # 靶点
    P2P = np.linalg.norm(PT - PO)  # 入皮点和靶点的距离
    seg = 3
    modName = "triPri%s" % i
    modN = "N"
    skin2bone = sRayCast(PO, PT, "Bonetempt")
    S2B = skin2bone[1]  # 入皮点到骨面的距离
    B2B = P2P - S2B  # 骨内长度
    SBArr = np.array([S2B, B2B])  # 皮骨长度数组
    SBArr = np.around(SBArr, 2)
    tableNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLTableNode")  #
    tableNode.SetName("SB_table%s" % i)
    slicer.util.updateTableFromArray(tableNode, tuple(SBArr))
    # AddPoint(PO) # 标记入皮点
    Cyl3 = p2pCyl(PT, PO, 10, modName, 2, seg, 0, "black", 0.6)  # 建一个3棱柱
    if i == 0:
      SBL = [SBArr]
    else:
      SBL = np.append(SBL, [SBArr], axis=0)
  return(P0,P1,SBL)

In [58]:
SBL = TriPrism()

In [60]:
SBL[2][0,0]

12.09

In [22]:
def DisTab(modName,PO):
  # PO = Pdata("T")
  seg = 3
  coord = Pcoord(modName)  # 得到3棱柱的坐标
  modNode = slicer.util.getNode(modName)
  slicer.mrmlScene.RemoveNode(modNode)  # 删掉3棱柱

  L = []
  XYZ = []
  for j in range(seg):  # 3棱柱坐标和模板的交点
    aaa = sRayCast(coord[2 * j], coord[2 * j + 1], "Skintempt")
    leng = aaa[1]
    XYZ = aaa[0]
    # AddPoint(XYZ)
    L = np.append(L, leng)  # 长度
    if j == 0:
      xyz = np.array([XYZ])
    else:
      xyz = np.append(xyz, np.array([XYZ]), axis=0)
  minL = min(L)  # 按每列求出长度最小值的索引
  min_index = np.argmin(L)  # ndarray最小值索引
  tabL = np.around(L - minL + 10, 2)
  tableNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLTableNode")
  tableNode.SetName("xyzL")
  slicer.util.updateTableFromArray(tableNode, tuple(tabL))
  ml = slicer.modules.markups.logic()
  Dpoint = xyz[min_index]
  Rpoint = xyz[min_index+1 if min_index != 2 else 0]
  Lpoint = xyz[min_index-1 if min_index != 0 else 2]


  newLandmarksNodeID = ml.AddNewFiducialNode("Clock")  # 交点成组
  AddPoint(Lpoint,color = "black")
  AddPoint(Dpoint,color = "red")
  AddPoint(Rpoint,color = "red")
  p2pCyl(Dpoint, PO, .2, "N", 10, seg)
  p2pCyl(PO, Dpoint, .2, "N", 10, seg)
  p2pCyl(Lpoint, Rpoint, .2, "N", 10, seg)
  p2pCyl(Rpoint, Lpoint, .2, "N", 10, seg)
  triangle3P(xyz[0],xyz[1],xyz[2])
  for ii, v in enumerate(xyz):
    fidNode = slicer.mrmlScene.GetNodeByID(newLandmarksNodeID)
    n = fidNode.AddFiducial(v[0],v[1],v[2])
    fidNode.SetNthFiducialLabel(n, "%s"%tabL[ii])
  return(xyz,tabL)

In [23]:
P0 = [-166.08001223158104,-168.6570030006596,16.874999999999996]

xyzL = DisTab("triPri1",P0)

MRMLNodeNotFoundException: could not find nodes in the scene by name or id 'triPri1'

In [21]:
xyzL

(array([[173.33372498, 163.57589722,  21.76644897],
       [160.11338806, 174.84855652,  21.98277092],
       [167.24505615, 169.99867249,   6.87577963]]), array([3.5362676 , 2.15904076, 3.77251653]))

### 保存和导入

In [24]:
import csv, codecs
import os
import io
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

Traceback (most recent call last):
  File "<console>", line 1, in <module>
ModuleNotFoundError: No module named 'pandas'
  File "<console>", line 1
    pip install pandas
              ^
SyntaxError: invalid syntax


In [4]:
renderWindow = slicer.app.layoutManager().threeDWidget(0).threeDView().renderWindow()
renderWindow.SetAlphaBitPlanes(1)
wti = vtk.vtkWindowToImageFilter()
wti.SetInputBufferTypeToRGBA()
wti.SetInput(renderWindow)
writer = vtk.vtkPNGWriter()
writer.SetFileName("shortcut/screenshot.png")
writer.SetInputConnection(wti.GetOutputPort())
writer.Write()

In [ ]:
renderWindow = slicer.app.layoutManager().threeDWidget(0).threeDView().renderWindow()
renderWindow.SetAlphaBitPlanes(1)
wti = vtk.vtkWindowToImageFilter()
wti.SetInputBufferTypeToRGBA()
wti.SetInput(renderWindow)
writer = vtk.vtkPNGWriter()
writer.SetFileName("c:/tmp/screenshot.png")
writer.SetInputConnection(wti.GetOutputPort())
writer.Write()

In [14]:
mrmlViewNode = threeDView.mrmlViewNode()
cameraNodes = slicer.util.getNodes( 'vtkMRMLCamera*' )
# cameraNode = None
for c in cameraNodes.items():
  cameraNode = c[1]
  if cameraNode.GetActiveTag() == mrmlViewNode.GetID():
    # found the cameraNode
    break
if not cameraNode:
  raise Exception( 'Something went terribly wrong..' )
camera = cameraNode.GetCamera()
cameraPosition = str( camera.GetPosition() )
cameraUp = str( camera.GetViewUp() )

NameError: name 'threeDView' is not defined

### gradeStep 

In [ ]:
def cropScrew(self,input, area):
  #Get bounds of screw
  bounds = input.GetPolyData().GetBounds()
  logging.debug(bounds)

  #Define bounds for cropping out the head or shaft of the screw respectively
  if area == 'head':
    i = bounds[2]
    j = bounds[3]-17
  elif area == 'shaft':
    i = bounds[3]-17
    j = bounds[3]

  #Create a box with bounds equal to that of the screw minus the head (-17)
  cropBox = vtk.vtkBox()
  cropBox.SetBounds(bounds[0],bounds[1],i,j,bounds[4],bounds[5])

  #Crop out head of screw
  extract = vtk.vtkExtractPolyDataGeometry()
  extract.SetImplicitFunction(cropBox)
  extract.SetInputData(input.GetPolyData())
  extract.Update()

  #PolyData of cropped screw
  output = extract.GetOutput()
  return output

In [81]:
P1 = data1[0,0]
P2 = data1[0,1]
p2pCyl(P2,P1,2,"axis",4,4,0,0,"green",1)
zuobiao = Pcoord("axis")
zhongdian = (zuobiao[0]+zuobiao[4])*.5 # 0和4是P1对角线坐标
P33 = zhongdian+([0,0,10])
p2pCyl(P2,P33,2,"axis",4,4,0,0,"green",1)